ΗΜΕΡΟΜΗΝΙΑ ΑΠΟ 26/01/2025-06/03/2025

In [1]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site και να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

In [2]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [8]:
page = 1
get_webpage_soup('https://www.tovima.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/')

https://www.tovima.gr/search/%CF%84%CE%B5%CE%BC%CF%80%CE%B7/ was scraped successfully


<!DOCTYPE html>
<html lang="el"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link href="https://www.tovima.gr/wp-content/cache/autoptimize/css/autoptimize_76662a222f33f67fe02f9b014b9c6e70.css" media="all" rel="stylesheet" type="text/css"/><title>τεμπη - ΤΟ ΒΗΜΑ</title><link href="https://www.tovima.gr/wp-content/themes/whsk_tovima.gr/common/imgs/iconsv_2/android-chrome-256x256.png" rel="shortcut icon"/><link href="https://www.tovima.gr/wp-content/themes/whsk_tovima.gr/common/imgs/iconsv_2/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="https://www.tovima.gr/wp-content/themes/whsk_tovima.gr/common/imgs/iconsv_2/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="https://www.tovima.gr/wp-content/themes/whsk_tovima.gr/common/imgs/iconsv_2/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="https://www.tovima.gr/wp-content/themes/whsk_tovima.gr/common/imgs/iconsv_2/si

In [9]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = [] # κενή λίστα, στην οποία θα κάνουμε append() το dictionary με τα data κάθε άρθρου
  for article in articlesL:
    try:
     title = article.find("h3", {"class" : "o-head f-400 my-0 is-size-2 zonabold"}).text.strip()
    except:
      title=""
    try:
      url = article.find("a", {"class": "columns is-mobile is-multiline"})["href"]
    except:
      url=""
    try:
      datetime_str = article.find("time").text.strip()
      date_str = datetime_str.split(",")[0]
      time_str = datetime_str.split(",")[1].strip()
    except:
      date_str = ""
      time_str = ""
    try:
      author = article.find("span", {"class":"vima-author line-height-1 manrope is-size-5 notblack-c f-700"}).text.strip()
    except:
      author = ""
    try:
      excert = article.find("p", {"class":"is-size-normal mb-0 mt-4 text-ellipsis text-ellipsis-4 grey-c line-height-1-3"}).text.strip()
    except:
      excert = ""

    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": url, "date": date_str, "time": time_str, "author": author, "description": excert}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

In [15]:
def scrape_to_vima_search_pages(search_item, start_page_nbr, end_page_nbr):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει

    url = f"https://www.tovima.gr/search/{search_item}/page/{page_nbr}//"

    print(f"\rScraping page {page_nbr}: {url}" + " "*20, end="")

    soup = get_webpage_soup(url)


    articlesL = soup.find_all("div", {"class":"column is-full wrap-category-row"})


    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    all_articles_dataL.extend(page_articles_dataL)


    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

In [16]:
#δοκιμή στην αναζήτηση για τον όρο "τέμπη"
scrape_to_vima_search_pages("τέμπη", 7, 17)

Scraping page 7: https://www.tovima.gr/search/τέμπη/page/7//                    https://www.tovima.gr/search/τέμπη/page/7// was scraped successfully
Scraping page 8: https://www.tovima.gr/search/τέμπη/page/8//                    https://www.tovima.gr/search/τέμπη/page/8// was scraped successfully
Scraping page 9: https://www.tovima.gr/search/τέμπη/page/9//                    https://www.tovima.gr/search/τέμπη/page/9// was scraped successfully
Scraping page 10: https://www.tovima.gr/search/τέμπη/page/10//                    https://www.tovima.gr/search/τέμπη/page/10// was scraped successfully
Scraping page 11: https://www.tovima.gr/search/τέμπη/page/11//                    https://www.tovima.gr/search/τέμπη/page/11// was scraped successfully
Scraping page 12: https://www.tovima.gr/search/τέμπη/page/12//                    https://www.tovima.gr/search/τέμπη/page/12// was scraped successfully
Scraping page 13: https://www.tovima.gr/search/τέμπη/page/13//                    https://www.tov

title  \
0    «Στέρεψαν τα δάκρυα και έγιναν οργή» – Συλλαλη...   
1    Ο Κασσελάκης συνυπέγραψε μήνυση κατά Χρυσοχοΐδ...   
2    Τέμπη - Κυβερνητικές πηγές: Αυτή είναι η αλήθε...   
3    Τέμπη: Νέα συλλαλητήρια την Παρασκευή - «Όλων ...   
4    Όσα έγιναν σε όλη τη χώρα στα νέα μαζικά συλλα...   
..                                                 ...   
215  5 το πρωί: Κοσμοσυρροή στις διαμαρτυρίες για τ...   
216  Τέμπη: Επαφή Φάμελλου με Ανδρουλάκη, Χαρίτση, ...   
217  Ο Φάμελλος φέρνει τον Μητσοτάκη στη Βουλή για ...   
218  Τέμπη: Η ανακοίνωση της Αστυνομίας για το συλλ...   
219  Καρυστιανού: «Ξέρετε πόσος χρόνος είναι τα 3 λ...   

                                           article_url        date   time  \
0    https://www.tovima.gr/2025/03/06/opinions/ster...  06.03.2025  21:13   
1    https://www.tovima.gr/2025/03/06/politics/temp...  06.03.2025  13:07   
2    https://www.tovima.gr/2025/03/06/politics/temp...  06.03.2025  12:40   
3    https://www.tovima.gr/2025/03/06/society/tempi...  06.03.2025  10:37   
4    https://www.tovima.gr/2025/03/05/society/tempi...  05.03.2025  22:31   
..                                                 ...         ...    ...   
215  https://www.tovima.gr/2025/01/27/5-to-proi/5-t...  27.01.2025  05:00   
216  https://www.tovima.gr/2025/01/26/politics/temp...  26.01.2025  22:05   
217  https://www.tovima.gr/2025/01/26/politics/o-fa...  26.01.2025  18:13   
218  https://www.tovima.gr/2025/01/26/society/tempi...  26.01.2025  17:53   
219  https://www.tovima.gr/2025/01/26/society/tempi...  26.01.2025  15:01   

                    author                                        description  
0    Δημήτρης Ελαφρόπουλος  Λίγες μέρες μετά τις μεγαλειώδεις συγκεντρώσει...  
1                           «Απαιτούμε κάμερες στις στολές των αστυνομικών...  
2                           Μαξίμου και Γεραπετρίτης απορρίπτουν τα περί e...  
3                           Στις 12 το μεσημέρι σε Προπύλαια κι 'Αγαλμα Βε...  
4                           Μαζικές συγκεντρώσεις για τους νεκρούς των Τεμ...  
..                     ...                                                ...  
215         Γιώτα Πεπαρίδη  Πλήθος κόσμου σε κάθε γωνιά της χώρας βγήκε στ...  
216                         Με αφορμή την τραγωδία στα Τέμπη, ο Σωκράτης Φ...  
217                         Προ ημερήσιας συζήτηση για τα Τέμπη ζητά ο ΣΥΡ...  
218                         Σε περίπου 30.000 άτομα ανήλθε η συμμετοχή στο...  
219                         Οσα δήλωσε η Μαρία Καρυστιανού στη συγκέντρωση...  

[220 rows x 6 columns]

In [17]:
to_vima_teasers_df=scrape_to_vima_search_pages("τέμπη", 7, 17)

Scraping page 7: https://www.tovima.gr/search/τέμπη/page/7//                    https://www.tovima.gr/search/τέμπη/page/7// was scraped successfully
Scraping page 8: https://www.tovima.gr/search/τέμπη/page/8//                    https://www.tovima.gr/search/τέμπη/page/8// was scraped successfully
Scraping page 9: https://www.tovima.gr/search/τέμπη/page/9//                    https://www.tovima.gr/search/τέμπη/page/9// was scraped successfully
Scraping page 10: https://www.tovima.gr/search/τέμπη/page/10//                    https://www.tovima.gr/search/τέμπη/page/10// was scraped successfully
Scraping page 11: https://www.tovima.gr/search/τέμπη/page/11//                    https://www.tovima.gr/search/τέμπη/page/11// was scraped successfully
Scraping page 12: https://www.tovima.gr/search/τέμπη/page/12//                    https://www.tovima.gr/search/τέμπη/page/12// was scraped successfully
Scraping page 13: https://www.tovima.gr/search/τέμπη/page/13//                    https://www.tov

In [22]:
to_vima_teasers_df.to_csv("/content/drive/MyDrive/to_vima_teasers.csv")

In [23]:
to_vima_teasers_df.to_csv('to_vima_teasers.csv', index=False)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
from google.colab import files
files.download('to_vima_teasers.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#articles scraping

In [25]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site ώστε να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

from tqdm import tqdm # για να βάλω progress bar με το οποίο θα παρακολουθώ την πορεία του scraping

In [26]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [32]:
to_vima_teasers_df = pd.read_csv('/content/drive/MyDrive/to_vima_teasers.csv')
to_vima_teasers_df.head()

Unnamed: 0                                              title  \
0           0  «Στέρεψαν τα δάκρυα και έγιναν οργή» – Συλλαλη...   
1           1  Ο Κασσελάκης συνυπέγραψε μήνυση κατά Χρυσοχοΐδ...   
2           2  Τέμπη - Κυβερνητικές πηγές: Αυτή είναι η αλήθε...   
3           3  Τέμπη: Νέα συλλαλητήρια την Παρασκευή - «Όλων ...   
4           4  Όσα έγιναν σε όλη τη χώρα στα νέα μαζικά συλλα...   

                                         article_url        date   time  \
0  https://www.tovima.gr/2025/03/06/opinions/ster...  06.03.2025  21:13   
1  https://www.tovima.gr/2025/03/06/politics/temp...  06.03.2025  13:07   
2  https://www.tovima.gr/2025/03/06/politics/temp...  06.03.2025  12:40   
3  https://www.tovima.gr/2025/03/06/society/tempi...  06.03.2025  10:37   
4  https://www.tovima.gr/2025/03/05/society/tempi...  05.03.2025  22:31   

                  author                                        description  
0  Δημήτρης Ελαφρόπουλος  Λίγες μέρες μετά τις μεγαλειώδεις συγκεντρώσει...  
1                    NaN  «Απαιτούμε κάμερες στις στολές των αστυνομικών...  
2                    NaN  Μαξίμου και Γεραπετρίτης απορρίπτουν τα περί e...  
3                    NaN  Στις 12 το μεσημέρι σε Προπύλαια κι 'Αγαλμα Βε...  
4                    NaN  Μαζικές συγκεντρώσεις για τους νεκρούς των Τεμ...

In [33]:
# εντοπίζω άρθρο για να πάρω τα στοιχεία του
article_url = to_vima_teasers_df.loc[0, "article_url"]
article_url

'https://www.tovima.gr/2025/03/06/opinions/sterepsan-ta-dakrya-kai-eginan-orgi-syllalitiria-gia-ta-tempi/'

In [34]:
soup = get_webpage_soup(article_url)

In [35]:
# βρίσκω το αρχικό block ενός ανοιγμένου άρθρου
article_block = soup.find('main', {"class" : "is-relative post-wrapper maxgridrow art-art01"})
print(article_block.prettify())

<main class="is-relative post-wrapper maxgridrow art-art01">
 <article class="columns is-relative is-variable is-4-desktop is-multiline">
  <div class="column is-full is-relative">
   <div class="wrap-article-header">
    <div class="columns is-multiline">
     <div class="column is-full is-relative">
      <div class="wrap-post-category is-relative">
       <span class="article-label uppercase">
        <a class="parent-category" href="https://www.tovima.gr/category/opinions/">
         Γνώμες
        </a>
       </span>
       <a class="is-hidden-small all-opinions" href="/category/opinions/" title="Κατηγορία Γνώμες">
        Όλα τα άρθρα
       </a>
      </div>
     </div>
     <div class="column is-full">
      <header class="article-header">
       <h1 class="post--title post-title zonareg is-size-0 is-size-2-mobile is-size-2-tablet f-400">
        «Στέρεψαν τα δάκρυα και έγιναν οργή» – Συλλαλητήρια για τα Τέμπη
       </h1>
       <div class="opinions-author mt-5 mb-2">
        

In [36]:
def get_article_elements_from_soup(soup, article_url):
  # απομόνωση του block με το περιεχόμενο του άρθρου απόο όλη τη σούπα της σελίδας
  article_block = soup.find('main', {"class" : "is-relative post-wrapper maxgridrow art-art01"})
  # article url
  article_url = article_url # το παίρνω από το arguement του function
  # title
  try:
    title = article_block.find('h1').text.strip()
  except:
    title=""
  # datetime
  try:
    date_str = article_block.find("time") ["datetime"]
    time_str = article_block.find("time", {'class' : 'line-height-1 post-date manrope lightgrey-c'}).text.strip()
  except:
    date_str = ""
    time_str = ""
  # section
  try:
    section = article_block.find('a', 'parent-category').text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("span", {"class": "vima-author"}).text.strip()
  except:
    author = ""
  # excerpt
  try:
    excert = article_block.find('h2').text.strip()
  except:
    excert=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", {"class": "post-body main-content pos-rel article-wrapper"}).find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
      text_body = " ".join(text_body_textsL)
      text_body
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = article_block.find("figure", {"class": "article-image"}).img["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = article_block.find('h2').text.strip()
  except:
    image_caption=""

  # Δημιουργία λεξικού με τα data του άρθρου
  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excert": excert, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_caption": image_caption}

  return articleD

In [37]:
def scrape_to_vima_articles_pages(to_vima_teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(to_vima_teasers_df.iterrows(), total=to_vima_teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [38]:
to_vima_articles_df = scrape_to_vima_articles_pages(to_vima_teasers_df)
to_vima_articles_df

Processing rows: 100%|██████████| 220/220 [11:38<00:00,  3.18s/it]



Scraping completed. Fetshed 220 articles.


article_url  \
0    https://www.tovima.gr/2025/03/06/opinions/ster...   
1    https://www.tovima.gr/2025/03/06/politics/temp...   
2    https://www.tovima.gr/2025/03/06/politics/temp...   
3    https://www.tovima.gr/2025/03/06/society/tempi...   
4    https://www.tovima.gr/2025/03/05/society/tempi...   
..                                                 ...   
215  https://www.tovima.gr/2025/01/27/5-to-proi/5-t...   
216  https://www.tovima.gr/2025/01/26/politics/temp...   
217  https://www.tovima.gr/2025/01/26/politics/o-fa...   
218  https://www.tovima.gr/2025/01/26/society/tempi...   
219  https://www.tovima.gr/2025/01/26/society/tempi...   

                                                 title        date   time  \
0    «Στέρεψαν τα δάκρυα και έγιναν οργή» – Συλλαλη...  06-03-2025  21:13   
1    Τέμπη: Ο Κασσελάκης συνυπέγραψε μήνυση κατά Χρ...  06-03-2025  13:07   
2    Τέμπη – Κυβερνητικές πηγές: Αυτή είναι η αλήθε...  06-03-2025  12:40   
3    Τέμπη: Νέα συλλαλητήρια την Παρασκευή από φοιτ...  06-03-2025  10:37   
4    Τέμπη: Όσα έγιναν στα συλλαλητήρια σε όλη τη χώρα  05-03-2025  22:31   
..                                                 ...         ...    ...   
215                                                                         
216  Τέμπη: Επαφή Φάμελλου με Ανδρουλάκη, Χαρίτση, ...  26-01-2025  22:05   
217  Ο Φάμελλος φέρνει τον Μητσοτάκη στη Βουλή για ...  26-01-2025  18:13   
218  Τέμπη: Η ανακοίνωση της Αστυνομίας για το συλλ...  26-01-2025  17:53   
219  Τέμπη – Καρυστιανού: «Ξέρετε πόσος χρόνος είνα...  26-01-2025  15:01   

      section                 author  \
0      Γνώμες  Δημήτρης Ελαφρόπουλος   
1    Πολιτική               Newsroom   
2    Πολιτική               Newsroom   
3    Κοινωνία               Newsroom   
4    Κοινωνία               Newsroom   
..        ...                    ...   
215                                    
216  Πολιτική               Newsroom   
217  Πολιτική               Newsroom   
218  Κοινωνία               Newsroom   
219  Κοινωνία               Newsroom   

                                                excert  \
0    Λίγες μέρες μετά τις μεγαλειώδεις συγκεντρώσει...   
1    «Απαιτούμε κάμερες στις στολές των αστυνομικών...   
2    Μαξίμου και Γεραπετρίτης απορρίπτουν τα περί e...   
3    Στις 12 το μεσημέρι σε Προπύλαια κι 'Αγαλμα Βε...   
4    Μαζικές συγκεντρώσεις για τους νεκρούς των Τεμ...   
..                                                 ...   
215                                                      
216  Με αφορμή την τραγωδία στα Τέμπη, ο Σωκράτης Φ...   
217  Προ ημερήσιας συζήτηση για τα Τέμπη ζητά ο ΣΥΡ...   
218  Σε περίπου 30.000 άτομα ανήλθε η συμμετοχή στο...   
219  Οσα δήλωσε η Μαρία Καρυστιανού στη συγκέντρωση...   

                                             text_body  \
0    Η μαζικότητα, η πολυμορφία και η ένταση του κό...   
1    Τη μήνυση κατά του Μιχάλη Χρυσοχοΐδη την οποία...   
2    «Το σημερινό δημοσίευμα της «Εστίας» για email...   
3    Με σχεδόν τις μισές προσαγωγές από τα χθεσινά ...   
4    Μετά τα μεγαλειώδη συλλαλητήρια για τους 57 νε...   
..                                                 ...   
215                                                      
216  Σύμφωνα με πληροφορίες, ο πρόεδρος του ΣΥΡΙΖΑ,...   
217  Σε συνεννόηση με τους αρχηγούς όλων των άλλων ...   
218  Ανακοίνωση εξέδωσε η ΕΛΑΣ σχετικά με τη μεγάλη...   
219  Πολυπληθείς διαδηλώσεις για τα θύματα στα Τέμπ...   

                                         feat_img_link  \
0    https://www.tovima.gr/wp-content/uploads/2025/...   
1    https://www.tovima.gr/wp-content/uploads/2025/...   
2    https://www.tovima.gr/wp-content/uploads/2025/...   
3    https://www.tovima.gr/wp-content/uploads/2025/...   
4    https://www.tovima.gr/wp-content/uploads/2025/...   
..                                                 ...   
215                                                      
216  https://www.tovima.gr/wp-content/uploads/2025/...   
217  https://www.tovima.

In [ ]:
# def clean text(text)
# cleaned_text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
# cleaned_text = " ".join(cleaned_text.split())
# return

## μπορούμε φτιάχοντας ένα τέτοιο def να καθαρίζουμε το κείμενο από κενά και άλλα στοιχεία που έχει μέσα

In [39]:
to_vima_articles_df.to_csv('/content/drive/MyDrive/to_vima_articles.csv')

In [40]:
to_vima_articles_df.to_csv('to_vima_articles.csv', index=False)

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
from google.colab import files
files.download('to_vima_articles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>